In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Jacobi and Gauss-Seidel 
We again consider the Poisson problem where we have the system of equations. We can write this equation as
\begin{align*}
    u_{ij} = \frac{1}{4}(u_{i-1,j} + u_{i+1,j} + u_{i, j-1} + u_{i, j+1}) - \frac{h^2}{4}f_{ij}
\end{align*}

This equation suggests the following iterative method to produce a new estimate $u^{[k+1]}$ from a current guess $u^{[k]}$:

\begin{align*}
    u_{ij}^{[k+1]} = \frac{1}{4}\left( u_{i-1,j}^{[k]} + u_{i+1,j}^{[k]} + u_{i,j-1}^{[k]} + u_{i,j+1}^{[k]}\right) 
                     - \frac{h^2}{4}f_{ij}
\end{align*}

In [76]:
m_1 = 10
h_1 = 1/(m_1+1)

def u_sol_1(x, y):
    return(np.sin(x) + np.sin(y))

def f_1(x, y):
    return(-np.sin(x) - np.sin(y))

In [63]:
def Poisson_GJ(u_sol, f, m, h, *arg):
    if arg == ():
         arg = math.floor(m**2*math.log(m))
    else:
        arg = arg[0]
            
    u_approx = np.ones((m + 2, m + 2))

    # Initializing the boundary terms with the boundary condition
    for i in range(m + 2):
        u_approx[0][i] = u_sol(0, i*h)
        u_approx[m+1][i] = u_sol(1, i*h)
        if i != 0 and i != m + 1:
            u_approx[i][0] = u_sol(0, i*h)
            u_approx[i][-1] = u_sol(1, i*h)


    u_new = np.copy(u_approx)

    for k in range(arg):
        for i in range(1, m + 1):
            for j in range(1, m + 1):
                u_new[i, j]=0.25*(u_approx[i-1][j]+u_approx[i+1][j]+u_approx[i][j-1]+u_approx[i][j+1] - 
                                  h*h*f(i*h,j*h))

        u_approx = np.copy(u_new)
        
    return(u_approx)

Instead of using a new matrix to store the updates value, we can store them in the same matrix and use those new updated values, therefore the new equation is
\begin{align*}
    u_{ij}^{[k+1]} = \frac{1}{4}\left(u_{i-1,j}^{[k + 1]} + u_{i+1,j}^{[k]} + u_{i,j-1}^{[k + 1]} + u_{i,j+1}^{[k]}                       \right) - \frac{h^2}{4}f_{ij}
\end{align*}

In [64]:
# Initializing the boundary terms with the boundary condition
    
def Poisson_GS(u_sol, f, m, h, *arg):
    
    if arg == ():
        arg = math.floor(m**2*math.log(m))
    else:
        arg = arg[0]
        
        
    u_approx = np.ones((m + 2, m + 2))
    
    for i in range(m + 2):
        u_approx[0][i] = u_sol(0, i*h)
        u_approx[m+1][i] = u_sol(1, i*h)
        if i != 0 and i != m + 1:
            u_approx[i][0] = u_sol(0, i*h)
            u_approx[i][-1] = u_sol(1, i*h)


    for k in range(arg):
        for i in range(1, m + 1):
            for j in range(1, m + 1):
                u_approx[i, j]=0.25*(u_approx[i-1][j]+u_approx[i+1][j]+u_approx[i][j-1]+u_approx[i][j+1] - 
                                     h*h*f(i*h,j*h))

    return(u_approx)

In [182]:
def Seidel(A, f, *arg):
    m = len(f)
    if arg == ():
        arg = math.floor(m**2*math.log(m))
    else:
        arg = arg[0]
    unknowns = np.zeros(m)
    
    for i in range(arg):
        for i in range(m):
            unknowns[i] = f[i]/A[i,i]
            for j in range(m):
                if j != i:
                    unknowns[i] -= (A[i,j]/A[i,i])*unknowns[j]
                    
    return(unknowns)
                
    

[1. 0.]


In [184]:
def Jacobi(A, f, *arg):
    m = len(f)
    if arg == ():
        arg = math.floor(m**2*math.log(m))
    else:
        arg = arg[0]
    unknowns = np.zeros(m)
    unknowns_1 = np.zeros(m)
    
    for i in range(arg):
        for i in range(m):
            unknowns_1[i] = f[i]/A[i,i]
            for j in range(m):
                if j != i:
                    unknowns_1[i] -= (A[i,j]/A[i,i])*unknowns[j]
                    
            unknows = np.copy(unknowns_1)
                    
    return(unknowns)

In [67]:
x = np.linspace(0, 1, m_1 + 2)
y = np.linspace(0, 1, m_1 + 2)
X, Y = np.meshgrid(x, y)

sol = u_sol_1(X, Y)

approx_GJ = Poisson_GJ(u_sol_1, f_1, m_1, h_1, 10)
approx_GS = Poisson_GS(u_sol_1, f_1, m_1, h_1, 10)
    
print('Error using Gauss Jacobi method',np.max(abs(approx_GJ - sol)))
    
print('Error using Gauss Seidel method',np.max(abs(approx_GS - sol)))

Error using Gauss Jacobi method 0.22543008659234853
Error using Gauss Seidel method 0.0780381195287998


# Successive overrelaxation
The Gauss-Seidel update moves $u_i$ in the right direction but is far too conservative in the amount allows $u_i$ to move. This suggest that we use the following two-stage update, illustrated again for the problem $u'' = f$:
\begin{align*}
    u_i^{\text{GS}} &= \frac{1}{2}\left( u_{i-1}^{[k+1]} + u_{i+1}^{[k]}  - h^2f_i \right), \\
    u_i^{[k+1]} &= u_i^{[k]} + \omega \left( u_i^{\text{GS}} - u_i^{[k]} \right)
\end{align*}

The formulas above can be combined to yield 
\begin{align*}
    u_i^{[k+1]} = \frac{\omega}{2} \left( u_{i-1}^{[k+1]} + u_{i+1}^{[k]} - h^2f_i \right) + (1-\omega)u_i^{[k]}.
\end{align*}

In [71]:
def sor_method(f, m, h, alpha, beta, *arg):
    if arg == ():
        arg = math.floor(m**2*math.log(m))
    else:
        arg = arg[0]
        
    u_approx = np.ones(m+2)
    u_approx[0] = alpha
    u_approx[m + 1] = beta
    
    omega = 2 - 2*np.pi*h
    
    for k in range(arg):
        for i in range(1, m + 1):
            u_approx[i] = (omega/2)*(u_approx[i-1] + u_approx[i+1] - h*h*f(i*h)) + (1-omega)*u_approx[i]
            
    return(u_approx)

In [77]:
def f_2(x):
    return(-np.sin(x))
alpha = np.sin(0)
beta = np.sin(1)

    
num_sol = sor_method(f_2, m_1, h_1, alpha, beta)

In [78]:
x = np.linspace(0, 1, m_1 + 2)
sol = np.sin(x)
print(np.max(abs(sol - num_sol)))

4.121713898164181e-05


# The method of steepest descent
This is a method to solve the general linear system $Au = f$.Here the method is first motivated as a descent method for solving a minimization problem. Consider the function $ \phi : \mathbb{R}^m \rightarrow \mathbb{R} $ defined by
\begin{align*}
    \phi(u) = \frac{1}{2}u^TAu-u^Tf.
\end{align*}
Let $u^*$ be the point where minimum occurs, this implies $ \nabla\phi(u^*) = 0$ and 
\begin{align*}
    \nabla\phi(u) = Au - f
\end{align*}

This implies that $u^*$ is the solution to the linear system $Au = f$.

From one estimate $u_{k-1}$ to $u^*$ we wish to obtain a better estimate $u_k$ by moving downhill, based on values of $\phi(u)$. Since the gradient vector $\nabla\phi(u)$ always points in the direction of most rapid increase of $\phi$. So we want to set
\begin{align*}
    u_k = u_{k-1} - \alpha_{k-1}\nabla\phi(u_{k-1})
\end{align*}
for some scalar $alpha_{k-1}$, chosen to solve the minimizaion problem
\begin{align*}
    \min_{\alpha \in \mathbb{R}} \phi(u_{k-1} - \alpha\nabla(u_{k-1}))
\end{align*}
Let $r_k = f - Au_k$ is called the residual vector based on the current approximation $u_k$. Solving the minimization problem for $\alpha$ we get
\begin{align*}
    \alpha = \frac{r^Tr}{r^TAr}
\end{align*}

This gives us the below algorithm \\

&nbsp; &nbsp; &nbsp; &nbsp; choose a guess $u_0$   
&nbsp; &nbsp; &nbsp; &nbsp; $r_0 = f-Au_0$         
&nbsp; &nbsp; &nbsp; &nbsp; for $ k = 1, 2, \ldots $  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $w_{k-1} = Ar_{k-1}$  

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $\alpha_{k-1} = (r^T_{k-1}r_{k-1})/(r^T_{k-1}w_{k-1})$ 

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $u_k = u_{k-1} + \alpha_{k-1}r_{k-1}$ 

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $r_k = r_{k-1} - \alpha_{k-1}w_{k-1}$   

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; if $\|{r_k}\|$ is less than some tolerance then stop

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; end

In [111]:
def steep_des(A, f):
    # Initial guess
    u_approx = np.ones(len(f))
    residual = f - np.dot(A, u_approx)
    while max(abs(residual)) >= pow(10, -3):
        w = np.dot(A, residual)
        print(w, residual)
        alpha = np.dot(residual, residual)/np.dot(residual, w)
        u_approx = u_approx + alpha*residual
        residual = residual - alpha*w
        
    
    return(u_approx)

In [114]:
steep_des([[2, 1], [1, 2]], [2, 0])

[-5. -7.] [-1. -3.]
[1.53846154 0.30769231] [ 0.92307692 -0.30769231]
[-0.87912088 -1.23076923] [-0.17582418 -0.52747253]
[0.27049873 0.05409975] [ 0.16229924 -0.05409975]
[-0.1545707  -0.21639899] [-0.03091414 -0.09274242]
[0.04756022 0.00951204] [ 0.02853613 -0.00951204]
[-0.02717727 -0.03804817] [-0.00543545 -0.01630636]
[0.00836224 0.00167245] [ 0.00501734 -0.00167245]
[-0.00477842 -0.00668979] [-0.00095568 -0.00286705]


array([ 1.3326472 , -0.66617657])

# The conjugate-gradient algorithm

In [126]:
def conjugate_grad(A, f):
    # Initial guess
    u_approx = np.zeros(len(f))
    residual = f - np.dot(A, u_approx)
    p_search = np.copy(residual)
    for i in range(len(f)):
        w = np.dot(A, p_search)
        alpha = np.dot(residual, residual)/np.dot(p_search, w)
        u_approx = u_approx + alpha * p_search
        residual_new = residual - alpha * w
        if np.max(abs(residual_new)) != 0:
            beta = np.dot(residual_new, residual_new)/np.dot(residual, residual)
            p_search = residual_new + beta * p_search
            residual = np.copy(residual_new)
        else:
            break
            
        
        
    return(u_approx)


In [127]:
conjugate_grad([[2, 1], [1, 2]], [2, 0])

array([ 1.33333333, -0.66666667])

# GMRES algorithm

In [441]:
def gmres(A, f):
    u_0 = np.zeros(len(f))
    r_0 = f - np.dot(A, u_0)
    
    Q = [r_0/np.linalg.norm(r_0)]
    Hess = np.zeros((len(f) + 1, len(f)))
    
    for k in range(len(f)):
        v = np.dot(A, Q[k])
        for i in range(k + 1):
            Hess[i, k] = np.dot(Q[i], v)
            v -= Hess[i, k]*Q[i]
           
        
        Hess[k + 1, k] = np.linalg.norm(v)
        if k != len(f) - 1:
            Q.append(v/Hess[k + 1, k])
            
        eta = np.zeros(k + 1)
        eta[0] = np.linalg.norm(r_0)
            
        y_k = np.linalg.lstsq(Hess[:k + 1, :k], eta)[0]
        
        
#         print(y_k, Q[:][:k+1], len(Q[0]))

        
        if np.linalg.norm(eta - np.dot(Hess[:k + 1, :k], y_k)) < pow(10, -3):
            return(u_0 + np.dot(np.transpose(Q[:k]), y_k))
        
        
        
    return(u_0 + np.dot(np.transpose(Q), y_k))
    
#     u_approx = np.zeros(len(f))
#     residual = f - np.dot(A, u_approx)
    
#     r_0 = np.linalg.norm(residual)
    
#     Q = [residual/r_0]
#     Hess = np.zeros((len(f) + 1, len(f)))
    
    
#     for k in range(1, len(f) + 1):
#         v = np.dot(A, Q[k - 1])
#         for i in range(k):
#             Hess[i, k - 1] = np.dot(Q[i], v)
#             v = v - Hess[i, k - 1]*Q[i]
#         Hess[k, k - 1] = np.linalg.norm(v)
#         if k != len(f):
#             Q.append(v/Hess[k, k - 1])
        
        
# #     print(Q, Hess)
        
#         eta = np.zeros(k + 1)
#         eta[0] = r_0
                
#         # Least square problem(Will write a program later)
#         y_k = np.linalg.lstsq(Hess[0:k + 1, 0:k], eta)[0]
        
#         print(Q, y_k)
        
#         if np.linalg.norm(eta - np.dot(Hess[0:k + 1, 0:k], y_k)) < pow(10, -3):
#             return(u_approx + np.dot(Q[:][:-1], y_k))
        
#     return(u_approx + np.dot(Q, y_k))



In [443]:
gmres(np.array([[2, 1, 0], [0, 1, 0], [0, 0, 1]]), [1, 2, 1])

/var/folders/q0/zx7hjt_j7bz80q29jndhn7b80000gn/T/ipykernel_21352/2053711827.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  y_k = np.linalg.lstsq(Hess[:k + 1, :k], eta)[0]


array([-0.5,  2. ,  1. ])

In [243]:
# def multi_grid(A, f, Coeff_matrix_gen):
#     if len(f) >= 3:
#         nu = random.randint(1, 10)
#         u_approx = Jacobi(A, f, nu)
#         residual = f - np.dot(A, u_approx)
#         course_residual = []
#         for i in range(math.floor((m - 1)/2)):
#             course_residual.append(u_approx[2*i])

#         course_A = Coeff_matrix_gen(math.floor((m - 1)/2))
        
    
#     elif len(f) == 2:
#         return(1/(A[0, 0]*A[1, 1] - A[0, 1]*A[1, 0])*np.dot([[A[1, 1], A[0, 0]], [-A[0, 1], -A[1, 0]]], f))
    
#     else:
#         return(1/A[0, 0]*f[0])